# OOP (2024 Fall) HW1: 불변 순서쌍을 활용해 가변 순서쌍 만들기 
- 이름: 오소민
- 학번: 20232397


-----
## Part A: 불변 순서쌍 Pair
1. 클래스(또는 정적) 팩토리 메소드 `of`를 를 수정하여 완성하라.
    - https://velog.io/@cjh8746/%EC%A0%95%EC%A0%81-%ED%8C%A9%ED%86%A0%EB%A6%AC-%EB%A9%94%EC%84%9C%EB%93%9CStatic-Factory-Method
1. 주석 부분을 풀어 제너릭 클래스인 `Pair`가 제너릭 인터페이스 `Comparable`를 구현하도록 완성하라.
    - https://docs.oracle.com/en/java/javase/22/docs/api/java.base/java/lang/Comparable.html

In [1]:
record Pair<F,S>(F first, S second) /* implements Comparable< ... > */ {
    
    // 아래 팩토리 메소드 of가 컴파일되지 않고 있으니 컴파일되도록 수정하라 (힌트: 제너릭 메소드)
    static Pair of (F first, S second) {
        return new Pair<>(first, second);
    }
    
    /*
    // 사전순이 되도록 정의하기. 즉 first가 더 큰 쪽이 더 크고, first가 같으면 second를 비교 ??동일하면?
    @Override
    public int compareTo(...) {
    }
    */
}

CompilerException: 

In [9]:
public record Pair<F, S>(F first, S second) implements Comparable<Pair<F, S>> {

    // 제네릭 메서드로 수정
    public static <F, S> Pair<F, S> of(F first, S second) {
        return new Pair<>(first, second);
    }

    // compareTo 메서드 구현
    @Override
    public int compareTo(Pair<F, S> other) {
        // first가 더 크면 양수, 같으면 second를 비교
        int firstComparison = ((Comparable<F>) this.first).compareTo(other.first);
        if (firstComparison != 0) {
            return firstComparison;
        }
        return ((Comparable<S>) this.second).compareTo(other.second);
    }
}

In [24]:
System.out.println(3)

In [ ]:
/* compareTo 가 잘 동작하는지
   - 왼쪽이 더 큰 경우
   - 왼쪽이 더 작은 경우
   - 같은 경우
   이런 경우를 각각 포함하도록 이렇게 최소 3개 이상의 테스트를 실행해 보라
*/

// 대략 이런 식으로 테스트 실행
// System.out.println( ??.compareTo(??) );
// System.out.println( ??.compareTo(??) );
// System.out.println( ??.compareTo(??) );

In [20]:
System.out.println(Pair.of(5, "apple").compareTo(Pair.of(3, "banana"))); // 출력: 양수 (5 > 3)
System.out.println(Pair.of(2, "apple").compareTo(Pair.of(4, "banana"))); // 출력: 음수 (2 < 4)
System.out.println(Pair.of(5, "apple").compareTo(Pair.of(5, "banana"))); // 출력: 음수 ("apple" < "banana")
System.out.println(Pair.of(5, "apple").compareTo(Pair.of(5, "apple"))); // 출력: 0 (완전히 동일)

In [25]:
public class Main {
    public static void main(String[] args) {
        // Pair 객체 생성
        Pair<Integer, String> pair1 = Pair.of(1, "apple");
        Pair<Integer, String> pair2 = Pair.of(2, "banana");
        Pair<Integer, String> pair3 = Pair.of(1, "banana");
        Pair<Integer, String> pair4 = Pair.of(1, "apple");

        // compareTo 테스트
        System.out.println(pair1.compareTo(pair2)); // -1 (1 < 2)
        System.out.println(pair1.compareTo(pair3)); // -1 ("apple" < "banana")
        System.out.println(pair1.compareTo(pair4)); // 0 (같은 Pair)
    }
}

------

In [3]:
class ModifiablePair<F,S> /* implements Comparable< ... > */ {
    private Pair<F,S> pair; // 이 field 말고 다른 field는 추가로 정의하지 말 것!

    ModifiablePair(F first, S second) {
        throw new UnsupportedOperationException("Not implemented yet");
    }
    
    // getter들은 pair에게 위임(delegate)하는 방식으로 작성하라
    public F first() {
        throw new UnsupportedOperationException("Not implemented yet");
    }
    public S second() {
        throw new UnsupportedOperationException("Not implemented yet");
    }
    
    // setter들은 적절히 정의하라
    public void setFirst(F first) {
        throw new UnsupportedOperationException("Not implemented yet");
    }
    public void setSecond(F second) {
        throw new UnsupportedOperationException("Not implemented yet");
    }

    /*
    // compareTo는 pair에게 위임(delegate)하는 방식으로 작성하라
    @Override
    public int compareTo(...) {
    }
    */

    @Override
    public String toString() { 
        return pair.toString(); // toString은 그냥 pair에게 위임
    }
}

CompilerException: 

In [21]:
class ModifiablePair<F extends Comparable<F>, S extends Comparable<S>> implements Comparable<ModifiablePair<F, S>> {
    private Pair<F, S> pair;

    // 생성자
    ModifiablePair(F first, S second) {
        this.pair = new Pair<>(first, second);
    }

    // getter들은 pair에게 위임
    public F first() {
        return pair.first();
    }

    public S second() {
        return pair.second();
    }

    // setter들은 새로운 값으로 새로운 Pair를 생성하여 할당
    public void setFirst(F first) {
        this.pair = new Pair<>(first, this.pair.second());
    }

    public void setSecond(S second) {
        this.pair = new Pair<>(this.pair.first(), second);
    }

    // compareTo는 pair에게 위임
    @Override
    public int compareTo(ModifiablePair<F, S> other) {
        return this.pair.compareTo(other.pair);
    }

    // toString은 pair에게 위임
    @Override
    public String toString() {
        return pair.toString();
    }
}

In [4]:
// ModifiablePair<F,S>의 getter와 setter들이 잘 동작하는지 확인할 수 있는 적절한 테스트 코드 작성하여 실행하라.
// 그러니까 `first()`, `second()`, `setFrist(F)`, `setSecond(S)` 메소드를 모두 활용하는 예시 코드를 작성해 실행하라는 말이다.

In [23]:
public class ModifiablePairTest {
    public static void main(String[] args) {
        // ModifiablePair 객체 생성
        ModifiablePair<Integer, String> modifiablePair = new ModifiablePair<>(1, "apple");

        // 초기 값 확인
        System.out.println("초기 값:");
        System.out.println("first: " + modifiablePair.first());   // 출력: 1
        System.out.println("second: " + modifiablePair.second()); // 출력: "apple"

        // 값 수정
        modifiablePair.setFirst(2);
        modifiablePair.setSecond("banana");

        // 수정된 값 확인
        System.out.println("수정된 값:");
        System.out.println("first: " + modifiablePair.first());   // 출력: 2
        System.out.println("second: " + modifiablePair.second()); // 출력: "banana"

        // toString 확인
        System.out.println("toString: " + modifiablePair); // 출력: Pair의 toString 결과
    }
}